# LangChain End-to-End: Prompt → Chain → RAG → Agent → UI

This single notebook merges the **basics** and **agent** demos into one coherent flow:

1. **Hello LLM** (baseline)  
2. **Prompting + LCEL + Output Parser**  
3. **RAG (build once, re-use)** with sources  
4. **Agent + Tools** (retriever tool + optional web search)  
5. **Tiny UI** (Gradio)  
6. **Custom Tool (@tool) example**  

> **Tip:** Open the notebook command palette and run all cells, or step through section by section.

## 0) Install & Configure (run once)

- Installs pinned versions to reduce API drift.  
- Prompts for your keys as needed.  
- **Optional**: If you have a LangSmith key, tracing will be enabled automatically.

> If you re-run the notebook later, you can skip re-installation if your environment already has these packages.

In [1]:
# Install LangChain & LangGraph v1 with compatible integrations
# Keep requests >= 2.32.5 to satisfy langchain-community (Colab may warn; it's OK)
%pip install -qU \
    "requests>=2.32.5" \
    "langchain>=1.0.3,<1.1" \
    "langgraph>=1.0,<2" \
    "langchain-openai>=1.0" \
    "langchain-community>=0.4,<1.0" \
    "langchain-text-splitters>=1.0.0" \
    beautifulsoup4 lxml faiss-cpu langchainhub tavily-python "gradio>=4.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take in

In [2]:
import importlib
def _ver(name):
    try:
        m = importlib.import_module(name)
        return getattr(m, "__version__", "n/a")
    except Exception as e:
        return f"not installed ({e})"
print("langchain           :", _ver("langchain"))
print("langgraph           :", _ver("langgraph"))
print("langchain-core      :", _ver("langchain_core"))
print("langchain-community :", _ver("langchain_community"))
print("langchain-openai    :", _ver("langchain_openai"))
print("langchainhub        :", _ver("langchainhub"))
print("langchain-text-splitters:", _ver("langchain_text_splitters"))
print("faiss-cpu           :", _ver("faiss"))
print("tavily-python       :", _ver("tavily"))


langchain           : 1.0.8
langgraph           : n/a
langchain-core      : 1.2.6
langchain-community : 0.4.1
langchain-openai    : n/a
langchainhub        : n/a


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

langchain-text-splitters: n/a
faiss-cpu           : 1.13.2
tavily-python       : n/a


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-p

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

### Loading Environment Variables from a `.env` File

While this notebook uses `os.getenv` and `getpass` to manage API keys, if you prefer using a `.env` file for local development or consistency, you can install and use the `python-dotenv` library.

In [4]:
# Install python-dotenv
%pip install python-dotenv

Next, create a `.env` file in the root of your Colab environment. You can use the `%%writefile` magic command for this. Replace `YOUR_VALUE_HERE` with your actual key or value.

In [5]:
from google.colab import userdata
import os

In [6]:
os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGSMITH_API_KEY')
os.environ["TAVILY_API_KEY"]=userdata.get('TAVILY_API_KEY')

In [7]:
# %%writefile .env
# OPENAI_API_KEY=OPENAI_API_KEY
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_API_KEY=LANGCHAIN_API_KEY
# TAVILY_API_KEY=TAVILY_API_KEY

In [8]:
# import os

# # Assuming .env is in the current working directory (root of Colab session)
# env_file_path = os.path.abspath('.env')
# print(f"The .env file is located at: {env_file_path}")

Now, load the environment variables from the `.env` file and access them using `os.getenv`:

In [9]:
import os
from dotenv import load_dotenv

# Load variables from .env file
# load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# print(f"OPENAI_API_KEY: {OPENAI_API_KEY}")
# print(f"LANGCHAIN_TRACING_V2: {LANGCHAIN_TRACING_V2}")
# print(f"LANGCHAIN_API_KEY: {LANGCHAIN_API_KEY}")
# print(f"TAVILY_API_KEY: {TAVILY_API_KEY}")

# You can then use these variables where needed, e.g., os.environ["OPENAI_API_KEY"] = my_api_key

In [10]:
import os
from typing import List, Any
from langchain.agents import create_agent
from langchain_core.tools import create_retriever_tool
try:
    from langchain_community.tools.tavily_search import TavilySearchResults
except Exception:
    TavilySearchResults = None

DEFAULT_MODEL = os.getenv("LC_V1_MODEL", "gpt-4o-mini")

def build_v1_agent(tools: List[Any], system_prompt: str = "You are a helpful assistant."):
    # In v1, `model` can be a string model id (e.g., 'gpt-4o-mini') or a chat model instance.
    return create_agent(model=DEFAULT_MODEL, tools=tools, system_prompt=system_prompt)

def run_agent(agent, question: str):
    try:
        return agent.invoke({"messages": [{"role": "user", "content": question}]})
    except Exception:
        return agent.invoke(question)


In [11]:

import os, getpass, warnings, sys

warnings.filterwarnings("ignore")

def ensure_env(key: str, prompt: str):
    if not os.getenv(key):
        try:
            val = getpass.getpass(prompt)
        except Exception:
            # Fallback for environments without stdin (e.g. some hosted notebooks)
            val = ""
        if val:
            os.environ[key] = val

# --- Required for LLM & embeddings ---
ensure_env("OPENAI_API_KEY", "Enter your OpenAI API Key (skipped if already set): ")

# --- Optional: LangSmith tracing ---
# If you have LANGCHAIN_API_KEY set, we turn on tracing v2 automatically.
if os.getenv("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    print("LangSmith tracing enabled (TRACING_V2=true).")
else:
    os.environ.pop("LANGCHAIN_TRACING_V2", None)
    print("LangSmith tracing not enabled (no LANGCHAIN_API_KEY). Proceeding without tracing.")

# --- Optional: Tavily web search ---
# If not set, we'll skip adding the Tavily tool; everything else runs fine.
if not os.getenv("TAVILY_API_KEY"):
    print("No TAVILY_API_KEY found. Agent will run without web search tool (retriever-only).")
else:
    print("Tavily web search tool will be available.")


LangSmith tracing enabled (TRACING_V2=true).
Tavily web search tool will be available.


## 1) Hello LLM (baseline)

A single LLM call; no structure, no grounding.  
We'll improve over this baseline throughout the notebook.

In [12]:
from langchain_openai import ChatOpenAI

# A deterministic model (temperature=0) for reproducible outputs.
llm = ChatOpenAI(temperature=0)

baseline_q = "In one sentence, how can LangSmith help with testing LLM apps?"
baseline_a = llm.invoke(baseline_q)
print("Q:", baseline_q)
print("\nBaseline (no context):\n", baseline_a.content)


Q: In one sentence, how can LangSmith help with testing LLM apps?

Baseline (no context):
 LangSmith can provide comprehensive testing services for LLM apps to ensure they meet quality standards and perform optimally.


## 2) Prompting + LCEL (LangChain Expression Language) + Output Parsing

Use **LCEL** (`|`) to pipe **PromptTemplate → LLM → OutputParser** so your code is composable and testable.

In [ ]:
sys_prompt = """

# Guideline
--------------
1.
2.
3.

# Task
classification of movie review

# Global Flows

# Do's

# Don't

# Output Requirements
-------
1. one word: positive, negative
2. The sentiment of the kv9e is:
3. {
  "movie":
  "sentiment:
}

"""

query: "avatar was the most amaxing "

In [13]:
from langchain_core.prompts import ChatPromptTemplate   # ✅ v1 path
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM (uses OPENAI_API_KEY from env)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt → LLM → Parser, piped with LCEL `|`
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise technical assistant."),
    ("human", "{question}")
])

chain = prompt | llm | StrOutputParser()

# Try it
chain.invoke({"question": "In one sentence, what does LCEL do?"})


'LCEL (Low Carbon Energy Limited) focuses on developing and implementing low-carbon energy solutions to reduce greenhouse gas emissions and promote sustainable energy practices.'

## 3) RAG

We’ll load a small corpus (LangSmith docs), split it, embed it, index it with **FAISS**, and wire a **Retrieval Chain**.

- This section runs **once** and is reused later by the Agent.
- If web loading fails, we fall back to a tiny local sample so the demo still runs.
- We'll also **surface sources** so you can see why answers improved.

In [17]:
# RAG (v1): Web loader → splitter → FAISS → retriever → LCEL chain
import os
os.environ.setdefault("USER_AGENT", "IK-LangChain-RAG/1.0 (contact: ops@your-org)")  # fixes the warning

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1) Load docs (pick any public pages you want indexed)
urls = [
    "https://python.langchain.com/docs/get_started/introduction/",
    "https://docs.smith.langchain.com/"
]
loader = WebBaseLoader(urls) # web scraping
docs = loader.load()

# 2) Chunk
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = splitter.split_documents(docs)

# 3) Embed & index
emb = OpenAIEmbeddings()  # uses OPENAI_API_KEY from env
vs = FAISS.from_documents(chunks, emb)
retriever = vs.as_retriever(search_kwargs={"k": 2})

# 4) Prompt (stuff-style: we inject all retrieved chunks into {context})
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise assistant. Use the provided CONTEXT to answer.\n"
     "If the answer isn't in the context, say you don't know.\n\nCONTEXT:\n{context}"),
    ("human", "{question}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# 5) LCEL pipeline: {question} flows through; {context} is produced by retriever
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 6) Try it
rag_chain.invoke("What is LangSmith and how does it relate to LangChain?")


'LangSmith is a platform that provides tools for developing, debugging, and deploying large language model (LLM) applications. It helps users trace requests, evaluate outputs, test prompts, and manage deployments in one place. LangSmith is framework agnostic, meaning it can be used with or without LangChain’s open-source libraries, which include langchain and langgraph. This allows users to prototype locally and then move to production with integrated monitoring and evaluation to build more reliable AI systems.'

In [18]:
retriever.invoke("What is LangSmith and how does it relate to LangChain?")
# .("What is LangSmith and how does it relate to LangChain")

[Document(id='8ae2cca8-b586-4382-b379-8b00b131168a', metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}, page_content='It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place.\nLangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries\nlangchain and langgraph.\nPrototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI systems.\nLangGraph Platform is now LangSmith Deployment. For more information, check out the Changelog.\n\u200bGet started\nCreate an accountSign up at smith.langchain.com (no credit card required).\nYou can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.\nCopy the key and save it securely.\nOnce your account and API key are ready, choose a quickstart to begin building with LangSmith:'),
 Document(id='e0c7f2

In [ ]:
# It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place.\nLangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries\nlangchain and langgraph.\nPrototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI systems.\nLangGraph Platform is now LangSmith Deployment. For more information, check out the Changelog.\n\u200bGet started\nCreate an accountSign up at smith.langchain.com (no credit card required).\nYou can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.\nCopy the key and save it securely.\nOnce your account and
# API key are ready, choose a quickstart to begin building with LangSmith:

## 4) Agent + Tools (on top of the same RAG)

We turn our retriever into a **Tool** and create an **OpenAI Functions Agent**.  
Optionally, if a **Tavily** API key is present, we add a web search tool.

> **Why an Agent?** It can decide *when* to use retrieval vs. answer directly, and sequence multi-step reasoning.

In [ ]:
# Tavily is a search engine specifically designed for AI agents and Large Language Models (LLMs).
# It focuses on providing real-time, accurate, and factual information for AI-driven applications.
# Unlike general-purpose search engines, Tavily prioritizes providing high-quality, concise, and readily usable data for AI to process.

# List of other tools - https://docs.langchain.com/oss/python/integrations/tools

In [19]:
# Agent + Tools (LangChain v1) — Structured JSON output

import os, json
from typing import Any, Dict, List, Optional
from pprint import pprint

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.tools import tool, create_retriever_tool
from langchain_core.messages import AIMessage, ToolMessage

# Optional: Tavily web search tool
try:
    from langchain_community.tools.tavily_search import TavilySearchResults
except Exception:
    TavilySearchResults = None

# ------------------ Define tools ------------------
tools: List[Any] = []

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

tools.append(add)

# add(5, 3)

# Add retriever tool if your RAG cell created `retriever`
if "retriever" in globals():
    kb_tool = create_retriever_tool(
        globals()["retriever"],
        name="kb_search",
        description="Search the indexed KB/curriculum docs and return relevant passages."
    )
    tools.append(kb_tool)

# Optional Tavily tool (requires TAVILY_API_KEY)
if TavilySearchResults and os.getenv("TAVILY_API_KEY"):
    tools.append(TavilySearchResults(max_results=5, include_answer=True))

# ------------------ Build agent ------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are a helpful technical assistant. Use tools when useful. "
        "Prefer kb_search for questions about our course/KB; use web search for general web info."
    ),
)

# ------------------ Helpers for structured output ------------------
def _final_text(res: Any) -> str:
    """Return final assistant text from AIMessage or from a {messages:[...]} dict."""
    if isinstance(res, AIMessage):
        return res.content or ""
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            if isinstance(m, AIMessage) or getattr(m, "type", "") == "ai":
                return getattr(m, "content", "") or ""
    return str(res)

def _collect_tool_calls_and_outputs(res: Any, max_len: int = 500) -> List[Dict[str, Any]]:
    """Return [{'name','args','output'}...] by pairing tool calls to ToolMessage outputs."""
    messages: List[Any] = []
    if isinstance(res, dict) and "messages" in res:
        messages = res["messages"]

    # Find first AI message with tool_calls (some runtimes store it in .tool_calls, others in additional_kwargs)
    tool_calls: List[Dict[str, Any]] = []
    for m in messages:
        if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
            tool_calls = m.tool_calls or []
            break
        addkw = getattr(m, "additional_kwargs", {}) if hasattr(m, "additional_kwargs") else {}
        if addkw.get("tool_calls"):
            tool_calls = addkw["tool_calls"]
            break

    # Map tool_call_id -> output from ToolMessage
    outputs: Dict[str, str] = {}
    for m in messages:
        if isinstance(m, ToolMessage):
            outputs[getattr(m, "tool_call_id", None)] = (getattr(m, "content", "") or "")

    def trunc(s: Optional[str]) -> str:
        if not s:
            return ""
        return s[:max_len] + ("…" if len(s) > max_len else "")

    structured: List[Dict[str, Any]] = []
    for c in tool_calls:
        structured.append({
            "name": c.get("name"),
            "args": c.get("args"),
            "output": trunc(outputs.get(c.get("id")))
        })
    return structured

def _usage(res: Any) -> Optional[Dict[str, Any]]:
    """Best-effort token usage extraction."""
    if isinstance(res, AIMessage):
        return getattr(res, "response_metadata", {}).get("token_usage")
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            meta = getattr(m, "response_metadata", {}) if hasattr(m, "response_metadata") else {}
            if meta.get("token_usage"):
                return meta["token_usage"]
    return None

def to_structured(res: Any) -> Dict[str, Any]:
    return {
        "answer": _final_text(res),
        "tools": _collect_tool_calls_and_outputs(res),
        "usage": _usage(res),
    }

# ------------------ Run and show structured JSON ------------------
query = "What is 41 + 1? Also, if I ask about LangGraph later, how would you use kb_search?"
res = agent.invoke({"messages": [{"role": "user", "content": query}]})
pprint(to_structured(res), width=100)


{'answer': 'The result of \\( 41 + 1 \\) is \\( 42 \\).\n'
           '\n'
           'Regarding LangGraph, if you ask about it later, I would use the `kb_search` function '
           'to look up relevant information in our knowledge base. This would allow me to provide '
           'you with accurate and detailed information about LangGraph based on the indexed '
           'documents.',
 'tools': [{'args': {'a': 41, 'b': 1}, 'name': 'add', 'output': '42.0'},
           {'args': {'query': 'LangGraph'},
            'name': 'kb_search',
            'output': 'of LangGraph. This allows us to take advantage of LangGraph’s durable '
                      'execution, human-in-the-loop support, persistence, and more.Learn moreDebug '
                      'with LangSmithGain deep visibility into complex agent behavior with '
                      'visualization tools that trace execution paths, capture state transitions, '
                      'and provide detailed runtime metrics.Learn mo

In [24]:
agent.invoke({"messages": [{"role": "user", "content": query}]})

{'messages': [HumanMessage(content='What is 41 + 1? Also, if I ask about LangGraph later, how would you use kb_search?', additional_kwargs={}, response_metadata={}, id='72140b0b-a1c5-4e28-9801-46f1b63e56e8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 192, 'total_tokens': 241, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CuNfNKd3QC8YoGMUoi9Ok9NBXvn5H', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8a64-22ea-7500-930d-8616eb7bbd8d-0', tool_calls=[{'name': 'add', 'args': {'a': 41, 'b': 1}, 'id': 'call_hi5bYHkhYA6Ef1OhqWYI4O21', 'type': 'tool_call'}, {'name': 'kb_search'

In [25]:
# Run multiple queries through the v1 agent and print tidy results

from pprint import pprint
from langchain_core.messages import AIMessage

queries = [
    "List two LangSmith capabilities that support evaluation and how to use them.",
    "Where do the docs explain tracing? Summarize in 3 bullets.",
]

def _final_text(res):
    if isinstance(res, AIMessage):
        return res.content or ""
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            if isinstance(m, AIMessage) or getattr(m, "type", "") == "ai":
                return getattr(m, "content", "") or ""
    return str(res)

for q in queries:
    print("\n" + "=" * 26)
    print("AGENT Q:", q)

    # v1 agents expect a messages list; fall back to raw string if needed
    try:
        res = agent.invoke({"messages": [{"role": "user", "content": q}]})
    except Exception:
        res = agent.invoke(q)

    # If you used Option 2 earlier, show structured JSON; else print final text
    if "to_structured" in globals():
        pprint(to_structured(res), width=100)
    else:
        print("\nAGENT A:\n", _final_text(res))



AGENT Q: List two LangSmith capabilities that support evaluation and how to use them.
{'answer': "It seems that I am unable to retrieve specific details about LangSmith's evaluation "
           'capabilities from the documentation. However, based on general knowledge, here are two '
           'common capabilities that support evaluation in platforms like LangSmith, along with '
           'how to use them:\n'
           '\n'
           '1. **Output Evaluation**:\n'
           '   - **Description**: This capability allows users to assess the quality and accuracy '
           'of the outputs generated by language models. It typically involves comparing the '
           "model's responses against expected results or benchmarks.\n"
           '   - **How to Use**: Users can input a set of prompts and their expected outputs into '
           "the evaluation tool. The system will then analyze the model's responses, providing "
           'metrics such as accuracy, relevance, and coherence

In [27]:
res = agent.invoke({"messages": [{"role": "user", "content": "what are some events happening in new york this weekend"}]})
pprint(to_structured(res), width=100)

{'answer': 'Here are some events happening in New York this weekend:\n'
           '\n'
           '1. **Concerts and Live Music**: Various artists will be performing across different '
           'venues. You can check out the lineup at places like Madison Square Garden and Barclays '
           'Center.\n'
           '\n'
           '2. **Theater Performances**: Broadway shows are always a highlight in New York. '
           'Popular shows like "The Lion King," "Wicked," and "Aladdin" are currently running.\n'
           '\n'
           '3. **Cirque du Soleil**: Their holiday show, "Twas the Night Before...", is returning '
           'to The Theater at Madison Square Garden.\n'
           '\n'
           '4. **Sports Events**: Catch a game at iconic venues like Yankee Stadium or Citi Field, '
           'depending on the season.\n'
           '\n'
           '5. **Comedy Shows**: Various comedy clubs will have performances throughout the '
           'weekend.\n'
           '\n'
   

## 5) Tiny UI (Gradio)

A minimal chat interface that routes user messages to the agent.  
If Tavily is not available, the agent still works with the retriever tool.

In [28]:

import gradio as gr
from langchain_core.messages import AIMessage

# -- helpers --
def _final_text(res):
    if isinstance(res, AIMessage):
        return res.content or ""
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            if isinstance(m, AIMessage) or getattr(m, "type", "") == "ai":
                return getattr(m, "content", "") or ""
    return str(res)

def _to_messages(history, message):
    # gr.ChatInterface history is List[Tuple[user, assistant]]
    msgs = []
    for u, a in history:
        if u: msgs.append({"role": "user", "content": u})
        if a: msgs.append({"role": "assistant", "content": a})
    msgs.append({"role": "user", "content": message})
    return msgs

def _ensure_agent():
    """Reuse global `agent` if defined; otherwise build a minimal one."""
    global agent
    try:
        agent  # already built in earlier cells
        return agent
    except NameError:
        from langchain_openai import ChatOpenAI
        from langchain.agents import create_agent
        from langchain_core.tools import tool

        @tool
        def add(a: float, b: float) -> float:
            "Add two numbers."
            return a + b

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        agent = create_agent(model=llm, tools=[add], system_prompt="You are helpful.")
        return agent

# -- chat function used by Gradio --
def chat_fn(message, history):
    try:
        ag = _ensure_agent()
        msgs = _to_messages(history, message)
        res = ag.invoke({"messages": msgs})  # v1 call
        return _final_text(res)
    except Exception as e:
        return f"Error: {e}"

# -- UI --
try:
    demo.close()  # close a previous demo if re-running in the same kernel
except Exception:
    pass

with gr.Blocks() as demo:
    gr.Markdown("# LangChain Agent Chat")
    gr.Markdown("Ask about your KB (kb_search) or general queries. Web search only if TAVILY_API_KEY is set.")
    gr.ChatInterface(chat_fn)
    gr.Markdown("Tip: Try “Where are tracing docs?” or “Multiply 3.5 and 4.”")

demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## 6) Custom Tool (@tool) example

One simple tool is enough to demonstrate schema and descriptions.  
The agent can call this tool if it detects a matching need.

## 7) Custom tools

Define new tools with the `@tool` decorator. Rebuild an agent by passing the updated tools list to `create_agent`, then invoke.


In [ ]:
# Custom tool (LangChain v1) — add `multiply`, rebuild agent, return structured JSON

import os, json
from typing import Any, Dict, List, Optional

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.tools import tool, create_retriever_tool
from langchain_core.messages import AIMessage, ToolMessage

# --- Define a custom tool ---
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers a and b."""
    return a * b

# --- Compose tools2 (reuse prior tools if they exist) ---
tools2: List[Any] = []
if "tools" in globals():  # from earlier cell
    tools2.extend(globals()["tools"])
tools2.append(multiply)

# Add retriever as a tool if your RAG cell created `retriever` and it's not already added
if "retriever" in globals() and not any(getattr(t, "name", "") == "kb_search" for t in tools2):
    tools2.append(create_retriever_tool(
        retriever,
        name="kb_search",
        description="Search the indexed KB/curriculum docs and return relevant passages."
    ))

# Optional Tavily search tool
try:
    from langchain_community.tools.tavily_search import TavilySearchResults
    if os.getenv("TAVILY_API_KEY") and not any(getattr(t, "name", "") == "tavily_search_results_json" for t in tools2):
        tools2.append(TavilySearchResults(max_results=5, include_answer=True))
except Exception:
    pass

# --- Build an agent (v1) ---
try:
    llm  # defined earlier?
except NameError:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

SYSTEM_PROMPT = (
    "You are a helpful technical assistant. Use tools when useful. "
    "Prefer kb_search for questions about our course/KB; use web search for general web info."
)
agent2 = create_agent(model=llm, tools=tools2, system_prompt=SYSTEM_PROMPT)

# --- Helpers to pretty-print structured JSON (answer + tools used + outputs) ---
def _final_text(res: Any) -> str:
    if isinstance(res, AIMessage):
        return res.content or ""
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            if isinstance(m, AIMessage) or getattr(m, "type", "") == "ai":
                return getattr(m, "content", "") or ""
    return str(res)

def _collect_tool_calls_and_outputs(res: Any, max_len: int = 500) -> List[Dict[str, Any]]:
    messages: List[Any] = res.get("messages", []) if isinstance(res, dict) else []

    # find tool calls
    tool_calls = []
    for m in messages:
        if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
            tool_calls = m.tool_calls or []
            break
        ak = getattr(m, "additional_kwargs", {}) if hasattr(m, "additional_kwargs") else {}
        if ak.get("tool_calls"):
            tool_calls = ak["tool_calls"]; break

    # map tool_call_id -> ToolMessage content
    outputs = {getattr(m, "tool_call_id", None): (getattr(m, "content", "") or "")
               for m in messages if isinstance(m, ToolMessage)}

    def trunc(s: Optional[str]) -> str:
        if not s: return ""
        return s[:max_len] + ("…" if len(s) > max_len else "")

    return [{"name": c.get("name"), "args": c.get("args"), "output": trunc(outputs.get(c.get("id")))}
            for c in tool_calls]

def _usage(res: Any) -> Optional[Dict[str, Any]]:
    if isinstance(res, AIMessage):
        return getattr(res, "response_metadata", {}).get("token_usage")
    if isinstance(res, dict) and "messages" in res:
        for m in reversed(res["messages"]):
            meta = getattr(m, "response_metadata", {}) if hasattr(m, "response_metadata") else {}
            if meta.get("token_usage"):
                return meta["token_usage"]
    return None

def to_structured(res: Any) -> Dict[str, Any]:
    return {"answer": _final_text(res), "tools": _collect_tool_calls_and_outputs(res), "usage": _usage(res)}

# --- Demo ---
q = "Multiply 3.5 by 4 and then list two LangSmith evaluation features."
res = agent2.invoke({"messages": [{"role": "user", "content": q}]})
print(json.dumps(to_structured(res), indent=2))


## 7) Wrap-up & Next steps

You built an end-to-end app:
- Baseline LLM → **Prompted chain** → **RAG** → **Agent with tools** → **(Optional) UI**
- Re-used the **same retriever** everywhere (built once).
- Optionally enabled **LangSmith** tracing for observability.

**Ideas to extend:**
- Swap FAISS for your vector DB of choice.  
- Add **validators** (output schemas) and **evaluation** suites.  
- Add domain-specific tools (databases, calculators, internal APIs).

> If you want to run the chat UI: uncomment `demo.launch()` in Section 5.